In [1]:
!pip install -U vnstock

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 2.7 MB/s eta 0:00:00


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 16.5 MB/s eta 0:00:00


In [3]:
pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=52e571d0bf5e757fa2ee8b8e181da5164528976d3153a4a95082a1a08d1655f2
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [4]:
pip install tensorflow

In [5]:
!pip install keras-self-attention

  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18895 sha256=8ff052290e72ba27b33b114e790d978f985888753ee688d6f3489e4965f385c6
  Stored in directory: /root/.cache/pip/wheels/46/f9/96/709295c836133071c12a300729fed4027757f889c01695feea
Successfully built keras-self-attention


In [6]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.1 MB/s eta 0:00:00


In [7]:
import numpy as np
import pandas as pd
from ta.momentum import RSIIndicator
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, RNN, concatenate
from tensorflow.keras.optimizers import RMSprop
from vnstock import *
from ta.volatility import BollingerBands
from tensorflow.keras.layers import Layer
import tensorflow as tf
from keras.layers import Input, Conv1D, MaxPooling1D, Bidirectional, Flatten, BatchNormalization
from sklearn.model_selection import RandomizedSearchCV
# from scikeras.wrappers import KerasClassifier

from ta.momentum import StochasticOscillator
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import CCIIndicator
from sklearn.preprocessing import StandardScaler
from ta.volume import ChaikinMoneyFlowIndicator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras import regularizers
from keras_self_attention import SeqSelfAttention
from tensorflow.keras.models import Model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.saving import register_keras_serializable
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [8]:
from vnstock import *
import pandas as pd
import os
from collections import Counter
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import RandomForestClassifier # Import RandomForestClassifier
from sklearn.metrics import accuracy_score # Import accuracy_score

In [9]:
from sklearn.linear_model import LinearRegression

In [10]:
pip install xgboost

In [11]:
from sklearn.metrics import log_loss

In [12]:
from xgboost import XGBClassifier

In [13]:
import optuna

In [14]:
import joblib

In [15]:
def load_stock(symbol, folder_path='stock_data'):
    """
    Load dữ liệu cổ phiếu từ mã cổ phiếu, sử dụng đường dẫn thư mục cố định.

    Parameters:
        symbol (str): Mã cổ phiếu (ví dụ: 'VCB')
        folder_path (str): Thư mục chứa file Excel (mặc định là 'stock_data')

    Returns:
        DataFrame chứa dữ liệu cổ phiếu.
    """
    file_path = os.path.join(folder_path, f"{symbol}.xlsx")

    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ Không tìm thấy file: {file_path}")

    try:
        df = pd.read_excel(file_path)
        print(f"✅ Đã load dữ liệu cho mã: {symbol}")
        return df
    except Exception as e:
        print(f"❌ Lỗi khi load dữ liệu {symbol}: {e}")
        return pd.DataFrame()

In [ ]:
df_vhm = load_stock('VNM','/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train')

✅ Đã load dữ liệu cho mã: VNM


In [17]:
df_vhm

,time,open,high,low,close,volume,P/B_Previous_Quarter,P/B_Same_Period_Last_Year,P/B_d_1,P/B_d_2,...,ROA (%)_Same_Period_Last_Year,ROA (%)_d_1,ROA (%)_d_2,ROA (%)_d_3,ROA (%)_d_4,ROA (%)_d_5,ROA (%)_d_6,ROA (%)_d_7,ROA (%)_d_8,ticket
0,2018-01-02,108.85,109.31,107.61,109.10,372760,11.487307,9.690078,11.487307,12.362129,...,0.328908,0.321509,0.334845,0.340996,0.343615,0.328908,0.330431,0.320700,0.305250,VNM
1,2018-01-03,110.14,110.91,109.41,110.34,557370,11.487307,9.690078,11.487307,12.362129,...,0.328908,0.321509,0.334845,0.340996,0.343615,0.328908,0.330431,0.320700,0.305250,VNM
2,2018-01-04,109.93,110.91,109.93,110.81,704350,11.487307,9.690078,11.487307,12.362129,...,0.328908,0.321509,0.334845,0.340996,0.343615,0.328908,0.330431,0.320700,0.305250,VNM
3,2018-01-05,110.39,110.91,109.62,110.14,518860,11.487307,9.690078,11.487307,12.362129,...,0.328908,0.321509,0.334845,0.340996,0.343615,0.328908,0.330431,0.320700,0.305250,VNM
4,2018-01-08,109.88,109.88,107.56,108.33,674800,11.487307,9.690078,11.487307,12.362129,...,0.328908,0.321509,0.334845,0.340996,0.343615,0.328908,0.330431,0.320700,0.305250,VNM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2023-12-25,63.52,64.55,63.52,63.80,2675110,4.581387,5.409208,4.581387,4.714668,...,0.167345,0.165294,0.164530,0.159914,0.167284,0.167345,0.178484,0.193378,0.206996,VNM
1496,2023-12-26,64.17,64.36,63.89,63.89,1977559,4.581387,5.409208,4.581387,4.714668,...,0.167345,0.165294,0.164530,0.159914,0.167284,0.167345,0.178484,0.193378,0.206996,VNM
1497,2023-12-27,64.37,64.56,63.89,63.89,1628408,4.581387,5.409208,4.581387,4.714668,...,0.167345,0.165294,0.164530,0.159914,0.167284,0.167345,0.178484,0.193378,0.206996,VNM
1498,2023-12-28,63.89,64.84,63.89,64.84,2410778,4.581387,5.409208,4.581387,4.714668,...,0.167345,0.165294,0.164530,0.159914,0.167284,0.167345,0.178484,0.193378,0.206996,VNM


In [18]:
def load_and_process_data(symbol, drop_na=True, file_path = ''):
    data = load_stock(symbol,file_path)
    if data.empty:
        return None

    df = data

    columns_to_drop = ['EV/EBITDA_Previous_Quarter','EV/EBITDA_Same_Period_Last_Year','ROIC (%)_Previous_Quarter','ROIC (%)_Same_Period_Last_Year']  # thay bằng danh sách cột bạn muốn xóa
    df.drop(columns=columns_to_drop, inplace=True)

     # Tính toán khối lượng trung bình
    volume_ma_period = 20  # Ví dụ: khối lượng trung bình 20 ngày
    df['volume_ma'] = df['volume'].rolling(window=volume_ma_period).mean()


    # Tính tỷ lệ khối lượng hiện tại / khối lượng trung bình
    df['volume_to_volume_ma_ratio'] = df['volume'] / df['volume_ma']


    # Tính toán EMA ngắn hạn (12 ngày)
    df['ema_12'] = df['close'].ewm(span=12, adjust=False).mean()


    # Tính toán EMA dài hạn (26 ngày)
    df['ema_26'] = df['close'].ewm(span=26, adjust=False).mean()

    # Tính toán SMA trung hạn (20 ngày)
    df['sma_20'] = df['close'].rolling(window=20).mean()

    # Tính toán SMA dài hạn (50 ngày)
    df['sma_50'] = df['close'].rolling(window=50).mean()

     # Tính toán ROC

    # df['roc_1'] = ((df['close'] - df['close'].shift(1)) / df['close'].shift(1)) * 100
    df['roc_5'] = ((df['close'] - df['close'].shift(5)) / df['close'].shift(5)) * 100
    # df['roc_9'] = ((df['close'] - df['close'].shift(9)) / df['close'].shift(9)) * 100

    # Tính toán %K (Stochastic Oscillator)
    stoch_period = 14
    df['%K'] = ((df['close'] - df['low'].rolling(window=stoch_period).min()) /
                (df['high'].rolling(window=stoch_period).max() - df['low'].rolling(window=stoch_period).min())) * 100

    # Tính toán %R (Williams %R)
    df['%R'] = ((df['high'].rolling(window=stoch_period).max() - df['close']) /
                (df['high'].rolling(window=stoch_period).max() - df['low'].rolling(window=stoch_period).min())) * -100

    # Tính toán CCI (Commodity Channel Index)
    cci_period = 20
    df['typical_price'] = (df['high'] + df['low'] + df['close']) / 3
    df['cci'] = CCIIndicator(high=df['high'], low=df['low'], close=df['close'], window=cci_period).cci()

    # Tính toán OBV (On Balance Volume)
    df['obv'] = OnBalanceVolumeIndicator(close=df['close'], volume=df['volume']).on_balance_volume()


    # Tính toán MACD
    df['macd'] = df['ema_12'] - df['ema_26']
    # Tính toán Signal line (EMA 9 ngày của MACD)
    df['signal_line'] = df['macd'].ewm(span=9, adjust=False).mean()
    # Tính toán MACD histogram
    df['macd_histogram'] = df['macd'] - df['signal_line']


    # Tính toán RSI
    rsi_period = 14
    df['rsi'] = RSIIndicator(df['close'], window=rsi_period).rsi()


    # Bollinger Bands
    indicator_bb = BollingerBands(close=df['close'], window=20, window_dev=2)
    df['bb_bbm'] = indicator_bb.bollinger_mavg()
    df['bb_bbh'] = indicator_bb.bollinger_hband()
    df['bb_bbl'] = indicator_bb.bollinger_lband()
    df['bb_bbp'] = indicator_bb.bollinger_pband()

    df['bb_bbh_bb_bbl_ratio'] = df['bb_bbh'] / df['bb_bbl']

    # =======================================================================

    # 1. Daily log return
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))

    # 2. Rolling volatility (biến động trượt)
    for window in [5, 10, 20, 30]:
        df[f'volatility_{window}d'] = df['log_return'].rolling(window=window).std()

    # 3. Rolling mean log return
    for window in [5, 10, 20, 30]:
        df[f'mean_log_return_{window}d'] = df['log_return'].rolling(window=window).mean()

    # 4. Sharpe-like ratio (mean return / volatility)
    for window in [5, 10, 20, 30]:
        mean_col = f'mean_log_return_{window}d'
        vol_col = f'volatility_{window}d'
        df[f'sharpe_like_{window}d'] = df[mean_col] / (df[vol_col] + 1e-9)

    # 5. Directional Feature - chuỗi ngày tăng liên tiếp
    df['up_streak'] = (df['log_return'] > 0).astype(int)
    df['up_streak'] = df['up_streak'] * (df['up_streak'].groupby((df['up_streak'] != df['up_streak'].shift()).cumsum()).cumcount() + 1)
    df['up_streak'] = df['up_streak'].where(df['log_return'] > 0, 0)

    # 6. Tỷ lệ ngày log return dương trong 20 ngày gần nhất
    df['pos_log_return_ratio_20d'] = df['log_return'].rolling(window=20).apply(lambda x: np.mean(x > 0), raw=True)

    # 7. Z-score của log return hiện tại
    for window in [5, 10, 20, 30]:
        mean_col = f'mean_log_return_{window}d'
        vol_col = f'volatility_{window}d'
        df[f'z_score_{window}d'] = (df['log_return'] - df[mean_col]) / (df[vol_col] + 1e-9)



    # Tính Annual Return (lợi nhuận hàng năm)
    df['annual_return'] = df['close'] / df['close'].shift(252) - 1  # 252 ngày giao dịch trong năm

    # Tính Volatility (Độ biến động) hàng năm (Standard Deviation of Returns)
    df['daily_return'] = df['close'].pct_change()  # Tỷ suất lợi nhuận hàng ngày
    df['annual_volatility'] = df['daily_return'].rolling(window=252).std() * (252 ** 0.5)  # Độ lệch chuẩn hàng năm (tính từ tỷ suất lợi nhuận hàng ngày)

    risk_free_rate = 0.03  # tương đương 3%/năm
    # Tính Sharpe ratio: (Rp - Rf) / Volatility
    df['sharpe_ratio'] = (df['annual_return'] - risk_free_rate) / df['annual_volatility']


    # Tính toán chỉ số cơ bản ==========================================================
    df['P/B_change_rate_flag'] = np.where(df['P/B_Same_Period_Last_Year'] == 0, 1, 0)
    df['P/B_change_rate'] = np.where(
        df['P/B_Same_Period_Last_Year'] != 0,
        (df['P/B_Previous_Quarter'] - df['P/B_Same_Period_Last_Year']) / df['P/B_Same_Period_Last_Year'],
        999
    )

    # P/E
    df['P/E_change_rate_flag'] = np.where(df['P/E_Same_Period_Last_Year'] == 0, 1, 0)
    df['P/E_change_rate'] = np.where(
        df['P/E_Same_Period_Last_Year'] != 0,
        (df['P/E_Previous_Quarter'] - df['P/E_Same_Period_Last_Year']) / df['P/E_Same_Period_Last_Year'],
        999
    )

    # P/S
    df['P/S_change_rate_flag'] = np.where(df['P/S_Same_Period_Last_Year'] == 0, 1, 0)
    df['P/S_change_rate'] = np.where(
        df['P/S_Same_Period_Last_Year'] != 0,
        (df['P/S_Previous_Quarter'] - df['P/S_Same_Period_Last_Year']) / df['P/S_Same_Period_Last_Year'],
        999
    )

    # P/Cash Flow
    df['P/Cash Flow_change_rate_flag'] = np.where(df['P/Cash Flow_Same_Period_Last_Year'] == 0, 1, 0)
    df['P/Cash Flow_change_rate'] = np.where(
        df['P/Cash Flow_Same_Period_Last_Year'] != 0,
        (df['P/Cash Flow_Previous_Quarter'] - df['P/Cash Flow_Same_Period_Last_Year']) / df['P/Cash Flow_Same_Period_Last_Year'],
        999
    )

    # EPS (VND)
    df['EPS (VND)_change_rate_flag'] = np.where(df['EPS (VND)_Same_Period_Last_Year'] == 0, 1, 0)
    df['EPS (VND)_change_rate'] = np.where(
        df['EPS (VND)_Same_Period_Last_Year'] != 0,
        (df['EPS (VND)_Previous_Quarter'] - df['EPS (VND)_Same_Period_Last_Year']) / df['EPS (VND)_Same_Period_Last_Year'],
        999
    )

    # BVPS (VND)
    df['BVPS (VND)_change_rate_flag'] = np.where(df['BVPS (VND)_Same_Period_Last_Year'] == 0, 1, 0)
    df['BVPS (VND)_change_rate'] = np.where(
        df['BVPS (VND)_Same_Period_Last_Year'] != 0,
        (df['BVPS (VND)_Previous_Quarter'] - df['BVPS (VND)_Same_Period_Last_Year']) / df['BVPS (VND)_Same_Period_Last_Year'],
        999
    )

    # ROE (%)
    df['ROE (%)_change_rate_flag'] = np.where(df['ROE (%)_Same_Period_Last_Year'] == 0, 1, 0)
    df['ROE (%)_change_rate'] = np.where(
        df['ROE (%)_Same_Period_Last_Year'] != 0,
        (df['ROE (%)_Previous_Quarter'] - df['ROE (%)_Same_Period_Last_Year']) / df['ROE (%)_Same_Period_Last_Year'],
        999
    )

    # ROA (%)
    df['ROA (%)_change_rate_flag'] = np.where(df['ROA (%)_Same_Period_Last_Year'] == 0, 1, 0)
    df['ROA (%)_change_rate'] = np.where(
        df['ROA (%)_Same_Period_Last_Year'] != 0,
        (df['ROA (%)_Previous_Quarter'] - df['ROA (%)_Same_Period_Last_Year']) / df['ROA (%)_Same_Period_Last_Year'],
        999
    )


    # Xứ lý với VNINDEX =============================================================================
    data_time = data['time']

    data_vnd = load_stock('VNINDEX', file_path)
    data_vnd = data_vnd.drop(columns=['ticket'])
    data_vnd.columns = ['time','open_vnd', 'high_vnd', 'low_vnd', 'close_vnd', 'volume_vnd']



    # Tính toán RSI
    rsi_period = 14
    data_vnd['rsi_vnd'] = RSIIndicator(data_vnd['close_vnd'], window=rsi_period).rsi()
    # Tính toán RSI-base-MA
    ma_period = 9
    data_vnd['rsi_base_ma_vnd'] = data_vnd['rsi_vnd'].rolling(window=ma_period).mean()
    data_vnd['rsi_rsi_base_ma_ratio_vnd'] = data_vnd['rsi_vnd'] / data_vnd['rsi_base_ma_vnd']

    # Tính toán khối lượng trung bình
    volume_ma_period = 20  # Ví dụ: khối lượng trung bình 20 ngày
    data_vnd['volume_ma_vnd'] = data_vnd['volume_vnd'].rolling(window=volume_ma_period).mean()
    # Tính tỷ lệ khối lượng hiện tại / khối lượng trung bình
    data_vnd['volume_to_volume_ma_ratio_vnd'] = data_vnd['volume_vnd'] / data_vnd['volume_ma_vnd']

    # Bollinger Bands
    indicator_bb = BollingerBands(close=data_vnd['close_vnd'], window=20, window_dev=2)
    data_vnd['bb_bbm_vnd'] = indicator_bb.bollinger_mavg()
    data_vnd['bb_bbh_vnd'] = indicator_bb.bollinger_hband()
    data_vnd['bb_bbl_vnd'] = indicator_bb.bollinger_lband()
    data_vnd['bb_bbp_vnd'] = indicator_bb.bollinger_pband()

    data_vnd['bb_bbh_bb_bbl_ratio_vnd'] = data_vnd['bb_bbh_vnd'] / data_vnd['bb_bbl_vnd']

    # Tính toán ROC
    roc_period = 9  # Chu kỳ 9 ngày, bạn có thể thay đổi tùy ý
    data_vnd['roc_vnd'] = ((data_vnd['close_vnd'] - data_vnd['close_vnd'].shift(roc_period)) / data_vnd['close_vnd'].shift(roc_period)) * 100

    # Tính toán %K (Stochastic Oscillator)
    stoch_period = 14
    data_vnd['%K_vnd'] = ((data_vnd['close_vnd'] - data_vnd['low_vnd'].rolling(window=stoch_period).min()) /
                (data_vnd['high_vnd'].rolling(window=stoch_period).max() - data_vnd['low_vnd'].rolling(window=stoch_period).min())) * 100

    # Tính toán %R (Williams %R)
    data_vnd['%R_vnd'] = ((data_vnd['high_vnd'].rolling(window=stoch_period).max() - data_vnd['close_vnd']) /
                (data_vnd['high_vnd'].rolling(window=stoch_period).max() - data_vnd['low_vnd'].rolling(window=stoch_period).min())) * -100

    # Tính toán CCI (Commodity Channel Index)
    cci_period = 20
    data_vnd['typical_price_vnd'] = (data_vnd['high_vnd'] + data_vnd['low_vnd'] + data_vnd['close_vnd']) / 3
    data_vnd['cci_vnd'] = CCIIndicator(high=data_vnd['high_vnd'], low=data_vnd['low_vnd'], close=data_vnd['close_vnd'], window=cci_period).cci()

    # Tính toán OBV (On Balance Volume)
    data_vnd['obv_vnd'] = OnBalanceVolumeIndicator(close=data_vnd['close_vnd'], volume=data_vnd['volume_vnd']).on_balance_volume()

    # Tính toán EMA ngắn hạn (12 ngày)
    data_vnd['ema_12_vnd'] = data_vnd['close_vnd'].ewm(span=12, adjust=False).mean()

    # Tính toán EMA dài hạn (26 ngày)
    data_vnd['ema_26_vnd'] = data_vnd['close_vnd'].ewm(span=26, adjust=False).mean()

    # Tính toán SMA trung hạn (20 ngày)
    data_vnd['sma_20_vnd'] = data_vnd['close_vnd'].rolling(window=20).mean()

    # Tính toán SMA dài hạn (50 ngày)
    data_vnd['sma_50_vnd'] = data_vnd['close_vnd'].rolling(window=50).mean()

    data_vnd['time'] = pd.to_datetime(data_vnd['time']).dt.date
    # merge
    df['time'] = pd.to_datetime(df['time']).dt.date
    df = pd.merge(df, data_vnd, on='time')


     # Tính tỷ lệ thay đổi giá
    threshold=0.01
    df['change'] = (df['close'].shift(-1) - df['close']) / df['close']

    # Gán nhãn
    df['target'] = 0  # Hold mặc định
    df.loc[df['change'] > threshold, 'target'] = 1   # Buy
    df.loc[df['change'] < -threshold, 'target'] = 2  # Sell

    if drop_na:
        df.dropna(inplace=True)

    return df

In [19]:
def add_slope_features(df, feature_base_names, n=8):
    """
    Với mỗi dòng trong df, tính hệ số gốc (slope) của đường hồi quy tuyến tính theo thời gian
    cho từng feature trong danh sách feature_base_names dựa trên các giá trị 8 quý gần nhất.

    Args:
        df (pd.DataFrame): DataFrame chứa các cột dạng 'feature_d_1' đến 'feature_d_n'.
        feature_base_names (list): Danh sách tên feature cơ bản (string).
        n (int): Số quý gần nhất để tính trend (mặc định 8).

    Returns:
        pd.DataFrame: DataFrame đã thêm các cột hệ số gốc tương ứng.
    """
    # Khởi tạo các cột coefficient_{feature} với NaN
    # for feature in feature_base_names:
    #     df[f"coefficient_{feature}"] = np.nan
    # Khởi tạo các cột coefficient_{feature} với NaN
    for feature in feature_base_names:
        col_name = f"coefficient_{feature}"
        if col_name not in df.columns:
            df[col_name] = np.nan

    # Duyệt từng hàng
    for index, row in df.iterrows():
        for feature in feature_base_names:
            y_values = []
            for i in range(n, 0, -1):  # d_8, ..., d_1
                val = row.get(f"{feature}_d_{i}", np.nan)
                y_values.append(val)

            y_values = np.array(y_values, dtype=float)

            # Cần ít nhất 2 điểm hợp lệ để fit mô hình
            if np.isnan(y_values).sum() > (n - 2):
                coef = np.nan
            else:
                X = np.arange(n).reshape(-1, 1)
                mask = ~np.isnan(y_values)
                model = LinearRegression()
                model.fit(X[mask], y_values[mask])
                coef = model.coef_[0]

            df.at[index, f"coefficient_{feature}"] = coef

    return df

In [20]:
def fn_load_and_process_data(symbol, drop_na=True, file_path = ''):
  df = load_and_process_data(symbol, drop_na, file_path)
  feature_base_names = ['P/B','P/E','P/S','P/Cash Flow','EPS (VND)','BVPS (VND)','ROE (%)','ROA (%)']
  df = add_slope_features(df, feature_base_names)
  return df

In [ ]:
df_vhm_process = fn_load_and_process_data("VHM", file_path= '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train')

✅ Đã load dữ liệu cho mã: VHM
✅ Đã load dữ liệu cho mã: VNINDEX


In [23]:
df_vhm_process

,time,open,high,low,close,volume,P/B_Previous_Quarter,P/B_Same_Period_Last_Year,P/B_d_1,P/B_d_2,...,change,target,coefficient_P/B,coefficient_P/E,coefficient_P/S,coefficient_P/Cash Flow,coefficient_EPS (VND),coefficient_BVPS (VND),coefficient_ROE (%),coefficient_ROA (%)
252,2019-05-22,62.76,64.00,62.76,62.98,702510,6.146763,0.000000,6.146763,6.972768,...,0.000000,0,-0.151429,3.593728,0.939508,3.523431,156.528796,2272.744777,0.058259,0.021232
253,2019-05-23,63.05,63.05,62.69,62.98,377180,6.146763,0.000000,6.146763,6.972768,...,-0.023182,2,-0.151429,3.593728,0.939508,3.523431,156.528796,2272.744777,0.058259,0.021232
254,2019-05-24,63.13,63.13,61.52,61.52,854870,6.146763,0.000000,6.146763,6.972768,...,-0.001138,0,-0.151429,3.593728,0.939508,3.523431,156.528796,2272.744777,0.058259,0.021232
255,2019-05-27,61.52,61.82,61.45,61.45,487560,6.146763,0.000000,6.146763,6.972768,...,-0.010578,2,-0.151429,3.593728,0.939508,3.523431,156.528796,2272.744777,0.058259,0.021232
256,2019-05-28,61.45,61.60,60.80,60.80,1436600,6.146763,0.000000,6.146763,6.972768,...,0.001151,0,-0.151429,3.593728,0.939508,3.523431,156.528796,2272.744777,0.058259,0.021232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1405,2023-12-22,40.20,40.40,40.00,40.30,3290547,1.012768,1.668381,1.012768,1.240452,...,0.022333,1,-0.165251,-0.413793,-0.284538,5.115060,123.741462,1848.338049,-0.007974,-0.009036
1406,2023-12-25,40.30,41.25,40.30,41.20,5960459,1.012768,1.668381,1.012768,1.240452,...,0.016990,1,-0.165251,-0.413793,-0.284538,5.115060,123.741462,1848.338049,-0.007974,-0.009036
1407,2023-12-26,41.50,42.20,41.30,41.90,10397855,1.012768,1.668381,1.012768,1.240452,...,0.000000,0,-0.165251,-0.413793,-0.284538,5.115060,123.741462,1848.338049,-0.007974,-0.009036
1408,2023-12-27,42.00,42.15,41.85,41.90,6037479,1.012768,1.668381,1.012768,1.240452,...,0.042959,1,-0.165251,-0.413793,-0.284538,5.115060,123.741462,1848.338049,-0.007974,-0.009036


In [ ]:
#  Hàm xử lý và chuẩn hóa dữ liệu
scaler = None
def process_and_create_data(symbol, features, target, time_steps=5, scaler=None, drop_na=True, file_path = '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train'):
    df = fn_load_and_process_data(symbol, drop_na, file_path= file_path)
    if df is None or len(df) < time_steps:
        return None, None  # Trả về None nếu dữ liệu không đủ dài
    if scaler is None:
        scaler = StandardScaler()
        df[features] = scaler.fit_transform(df[features])
    else:
        df[features] = scaler.transform(df[features])

    X = df[features]
    y = df[target]
    return X, y, scaler

In [ ]:
vn30_symbols = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
# features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','rsi_vnd','rsi_base_ma_vnd','rsi_rsi_base_ma_ratio_vnd','volume_ma_vnd','volume_to_volume_ma_ratio_vnd','bb_bbm_vnd','bb_bbh_vnd','bb_bbl_vnd','bb_bbp_vnd','bb_bbh_bb_bbl_ratio_vnd','roc_vnd','%K_vnd','%R_vnd','cci_vnd','obv_vnd','ema_12_vnd','ema_26_vnd','sma_20_vnd','sma_50_vnd','P/B_Previous_Quarter', 'P/B_change_rate','P/B_change_rate_flag','P/E_Previous_Quarter','P/E_change_rate','P/E_change_rate_flag','P/S_Previous_Quarter','P/S_change_rate','P/S_change_rate_flag','P/Cash Flow_Previous_Quarter','P/Cash Flow_change_rate','P/Cash Flow_change_rate_flag','EPS (VND)_Previous_Quarter','EPS (VND)_change_rate', 'EPS (VND)_change_rate_flag','BVPS (VND)_Previous_Quarter','BVPS (VND)_change_rate', 'BVPS (VND)_change_rate_flag','ROE (%)_Previous_Quarter','ROE (%)_change_rate','ROE (%)_change_rate_flag','ROA (%)_Previous_Quarter','ROA (%)_change_rate','ROA (%)_change_rate_flag']
features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','rsi_vnd','rsi_base_ma_vnd','rsi_rsi_base_ma_ratio_vnd','volume_ma_vnd','volume_to_volume_ma_ratio_vnd','bb_bbm_vnd','bb_bbh_vnd','bb_bbl_vnd','bb_bbp_vnd','bb_bbh_bb_bbl_ratio_vnd','roc_vnd','%K_vnd','%R_vnd','cci_vnd','obv_vnd','ema_12_vnd','ema_26_vnd','sma_20_vnd','sma_50_vnd','P/B_Previous_Quarter', 'P/B_change_rate','P/B_change_rate_flag','P/E_Previous_Quarter','P/E_change_rate','P/E_change_rate_flag','P/S_Previous_Quarter','P/S_change_rate','P/S_change_rate_flag','P/Cash Flow_Previous_Quarter','P/Cash Flow_change_rate','P/Cash Flow_change_rate_flag','EPS (VND)_Previous_Quarter','EPS (VND)_change_rate', 'EPS (VND)_change_rate_flag','BVPS (VND)_Previous_Quarter','BVPS (VND)_change_rate', 'BVPS (VND)_change_rate_flag','ROE (%)_Previous_Quarter','ROE (%)_change_rate','ROE (%)_change_rate_flag','ROA (%)_Previous_Quarter','ROA (%)_change_rate','ROA (%)_change_rate_flag','log_return','volatility_5d','volatility_10d','volatility_20d','volatility_30d','mean_log_return_5d','mean_log_return_10d','mean_log_return_20d','mean_log_return_30d','sharpe_like_5d','sharpe_like_10d','sharpe_like_20d','sharpe_like_30d','up_streak','pos_log_return_ratio_20d','z_score_5d','z_score_10d','z_score_20d','z_score_30d','annual_return','daily_return','sharpe_ratio','coefficient_P/B','coefficient_P/E','coefficient_P/S','coefficient_P/Cash Flow','coefficient_EPS (VND)','coefficient_BVPS (VND)','coefficient_ROE (%)','coefficient_ROA (%)','distance_to_nearest_quarter']
target = 'target'
X_train_all, y_train_all = [], []
X_val_all, y_val_all = [], []

for symbol in vn30_symbols:
    X, y, scaler = process_and_create_data(symbol, features, target, 5, scaler,
                                           file_path='/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_train')
    if X is not None and y is not None:
        n = len(X)
        split_index = int(n * 0.8)

        X_train_all.append(X[:split_index])
        y_train_all.append(y[:split_index])
        X_val_all.append(X[split_index:])
        y_val_all.append(y[split_index:])

# Kết hợp lại thành các tập tổng
X_train = np.concatenate(X_train_all, axis=0)
y_train = np.concatenate(y_train_all, axis=0)
X_val = np.concatenate(X_val_all, axis=0)
y_val = np.concatenate(y_val_all, axis=0)

✅ Đã load dữ liệu cho mã: ACB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BCM
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BID
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BVH
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: CTG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: FPT
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: GAS
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: GVR
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: HDB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: HPG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: LPB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MBB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MSN
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MWG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: PLX
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: SAB
✅ Đã load 

divide by zero encountered in log


✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: VIC
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: VJC
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: VNM
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: VPB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: VRE
✅ Đã load dữ liệu cho mã: VNINDEX


In [ ]:
X_train.shape

(28727, 94)

In [ ]:
counts = Counter(y_train)

print(f"Số lượng 2 (Sell): {counts[2]}")
print(f"Số lượng  0 (Hold): {counts[0]}")
print(f"Số lượng  1 (Buy):  {counts[1]}")

Số lượng 2 (Sell): 6960
Số lượng  0 (Hold): 14235
Số lượng  1 (Buy):  7532


In [ ]:
# Load dữ liệu test
# Xử lý và kết hợp dữ liệu cho nhiều cổ phiếu để tạo tập dữ liệu
# vn30_symbols = ["ACB","BCM","BID","BVH","CTG","FPT","GAS","GVR","HDB","HPG","MBB","MSN","MWG","PLX","POW","SAB","SHB","SSB","SSI","STB","TCB","TPB","VCB","VHM","VIB","VIC","VJC","VNM","VPB","VRE"]
vn30_symbols = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']

# Đặc trưng và biến mục tiêu
# features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','rsi_vnd','rsi_base_ma_vnd','rsi_rsi_base_ma_ratio_vnd','volume_ma_vnd','volume_to_volume_ma_ratio_vnd','bb_bbm_vnd','bb_bbh_vnd','bb_bbl_vnd','bb_bbp_vnd','bb_bbh_bb_bbl_ratio_vnd','roc_vnd','%K_vnd','%R_vnd','cci_vnd','obv_vnd','ema_12_vnd','ema_26_vnd','sma_20_vnd','sma_50_vnd','P/B_Previous_Quarter', 'P/B_change_rate','P/B_change_rate_flag','P/E_Previous_Quarter','P/E_change_rate','P/E_change_rate_flag','P/S_Previous_Quarter','P/S_change_rate','P/S_change_rate_flag','P/Cash Flow_Previous_Quarter','P/Cash Flow_change_rate','P/Cash Flow_change_rate_flag','EPS (VND)_Previous_Quarter','EPS (VND)_change_rate', 'EPS (VND)_change_rate_flag','BVPS (VND)_Previous_Quarter','BVPS (VND)_change_rate', 'BVPS (VND)_change_rate_flag','ROE (%)_Previous_Quarter','ROE (%)_change_rate','ROE (%)_change_rate_flag','ROA (%)_Previous_Quarter','ROA (%)_change_rate','ROA (%)_change_rate_flag']
# features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','rsi_vnd','rsi_base_ma_vnd','rsi_rsi_base_ma_ratio_vnd','volume_ma_vnd','volume_to_volume_ma_ratio_vnd','bb_bbm_vnd','bb_bbh_vnd','bb_bbl_vnd','bb_bbp_vnd','bb_bbh_bb_bbl_ratio_vnd','roc_vnd','%K_vnd','%R_vnd','cci_vnd','obv_vnd','ema_12_vnd','ema_26_vnd','sma_20_vnd','sma_50_vnd']
features = ['volume_ma','volume_to_volume_ma_ratio','ema_12','ema_26','sma_20','sma_50','roc_5','%K','%R','cci','obv','macd','signal_line','macd_histogram','rsi','bb_bbm','bb_bbh','bb_bbl','bb_bbp','bb_bbh_bb_bbl_ratio','rsi_vnd','rsi_base_ma_vnd','rsi_rsi_base_ma_ratio_vnd','volume_ma_vnd','volume_to_volume_ma_ratio_vnd','bb_bbm_vnd','bb_bbh_vnd','bb_bbl_vnd','bb_bbp_vnd','bb_bbh_bb_bbl_ratio_vnd','roc_vnd','%K_vnd','%R_vnd','cci_vnd','obv_vnd','ema_12_vnd','ema_26_vnd','sma_20_vnd','sma_50_vnd','P/B_Previous_Quarter', 'P/B_change_rate','P/B_change_rate_flag','P/E_Previous_Quarter','P/E_change_rate','P/E_change_rate_flag','P/S_Previous_Quarter','P/S_change_rate','P/S_change_rate_flag','P/Cash Flow_Previous_Quarter','P/Cash Flow_change_rate','P/Cash Flow_change_rate_flag','EPS (VND)_Previous_Quarter','EPS (VND)_change_rate', 'EPS (VND)_change_rate_flag','BVPS (VND)_Previous_Quarter','BVPS (VND)_change_rate', 'BVPS (VND)_change_rate_flag','ROE (%)_Previous_Quarter','ROE (%)_change_rate','ROE (%)_change_rate_flag','ROA (%)_Previous_Quarter','ROA (%)_change_rate','ROA (%)_change_rate_flag','log_return','volatility_5d','volatility_10d','volatility_20d','volatility_30d','mean_log_return_5d','mean_log_return_10d','mean_log_return_20d','mean_log_return_30d','sharpe_like_5d','sharpe_like_10d','sharpe_like_20d','sharpe_like_30d','up_streak','pos_log_return_ratio_20d','z_score_5d','z_score_10d','z_score_20d','z_score_30d','annual_return','daily_return','sharpe_ratio','coefficient_P/B','coefficient_P/E','coefficient_P/S','coefficient_P/Cash Flow','coefficient_EPS (VND)','coefficient_BVPS (VND)','coefficient_ROE (%)','coefficient_ROA (%)','distance_to_nearest_quarter']
target = 'target'
# Khởi tạo danh sách để lưu dữ liệu
X_all, y_all = [], []
# scaler = None

for symbol in vn30_symbols:
    X, y, scaler = process_and_create_data(symbol, features, target,5, scaler, file_path = '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/data_test')
    if X is not None and y is not None:
        X_all.append(X)
        y_all.append(y)

# Kết hợp tất cả dữ liệu lại
X_test = np.concatenate(X_all, axis=0)
y_test = np.concatenate(y_all, axis=0)



✅ Đã load dữ liệu cho mã: ACB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BCM
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BID
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: BVH
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: CTG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: FPT
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: GAS
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: GVR
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: HDB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: HPG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: LPB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MBB
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MSN
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: MWG
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: PLX
✅ Đã load dữ liệu cho mã: VNINDEX
✅ Đã load dữ liệu cho mã: SAB
✅ Đã load 

In [ ]:
X_test.shape

(7380, 94)

In [ ]:
counts = Counter(y_test)

print(f"Số lượng 2 (Sell): {counts[2]}")
print(f"Số lượng  0 (Hold): {counts[0]}")
print(f"Số lượng  1 (Buy):  {counts[1]}")

Số lượng 2 (Sell): 1302
Số lượng  0 (Hold): 4656
Số lượng  1 (Buy):  1422


In [ ]:
# Tính trọng số cân bằng
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

print("Class Weights:", class_weight_dict)

Class Weights: {np.int64(0): np.float64(0.6726846973422316), np.int64(1): np.float64(1.2713312090635511), np.int64(2): np.float64(1.3758141762452107)}


In [ ]:
# Dùng class_weight
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weight_dict)

In [ ]:
rf.fit(X_train, y_train)

RandomForestClassifier(class_weight={np.int64(0): np.float64(0.6726846973422316),
                                     np.int64(1): np.float64(1.2713312090635511),
                                     np.int64(2): np.float64(1.3758141762452107)},
                       random_state=42)

In [ ]:
y_pred_rf = rf.predict(X_test)

In [ ]:
print("Rf Accuracy:", accuracy_score(y_test, y_pred_rf))

Rf Accuracy: 0.6245257452574525


In [ ]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rf))

Confusion Matrix:
[[4502  109   45]
 [1300   88   34]
 [1227   56   19]]


In [ ]:
print("Classification Report:")
print(classification_report(y_test, y_pred_rf))

Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.97      0.77      4656
           1       0.35      0.06      0.11      1422
           2       0.19      0.01      0.03      1302

    accuracy                           0.62      7380
   macro avg       0.39      0.35      0.30      7380
weighted avg       0.51      0.62      0.51      7380



In [ ]:
def objective(trial):
    params = {
        "n_estimators": 1000,
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "use_label_encoder": False,
        "eval_metric": "mlogloss",
        "random_state": 42,
    }

    model = XGBClassifier(**params)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    preds_proba = model.predict_proba(X_val)
    loss = log_loss(y_val, preds_proba)
    return loss

In [ ]:
study = optuna.create_study(direction="minimize")

[I 2025-04-25 03:23:17,554] A new study created in memory with name: no-name-0fed03f1-87d0-4c75-adaa-9ab6b871cc97


In [ ]:
study.optimize(objective, n_trials=50)

In [ ]:
print("Best trial:", study.best_trial.params)

Best trial: {'max_depth': 7, 'learning_rate': 0.010106271727691464, 'subsample': 0.7256796159424765, 'colsample_bytree': 0.7266740288520646}


In [ ]:
best_params = study.best_trial.params

In [ ]:
best_model = XGBClassifier(
    n_estimators=1000,  # giữ nguyên như trong objective
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42,
    **best_params
)

In [ ]:
best_model.fit(X_train, y_train)

[05:09:58] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7266740288520646, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.010106271727691464,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_pred_best_model = best_model.predict(X_test)

In [ ]:
print("🎯 Accuracy:", accuracy_score(y_test, y_pred_best_model))

🎯 Accuracy: 0.6212737127371274


In [ ]:
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_best_model))


📊 Confusion Matrix:
[[4406  210   40]
 [1255  149   18]
 [1144  128   30]]


In [ ]:
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred_best_model))


📋 Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.95      0.77      4656
           1       0.31      0.10      0.16      1422
           2       0.34      0.02      0.04      1302

    accuracy                           0.62      7380
   macro avg       0.43      0.36      0.32      7380
weighted avg       0.53      0.62      0.52      7380



In [ ]:
best_model_v2 = XGBClassifier(
    n_estimators=1000,  # giữ nguyên như trong objective
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42,
    **best_params
)

In [ ]:
# Hoặc nếu muốn kết hợp thêm val:
X_train_full = np.concatenate([X_train, X_val], axis=0)
y_train_full = np.concatenate([y_train, y_val], axis=0)
best_model_v2.fit(X_train_full, y_train_full)

[05:25:16] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7266740288520646, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', feature_types=None, feature_weights=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.010106271727691464,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, ...)

In [ ]:
y_pred_best_model_2 = best_model_v2.predict(X_test)

In [ ]:
print("🎯 Accuracy:", accuracy_score(y_test, y_pred_best_model_2))

🎯 Accuracy: 0.6285907859078591


In [ ]:
print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_best_model_2))


📊 Confusion Matrix:
[[4471  142   43]
 [1265  128   29]
 [1168   94   40]]


In [ ]:
print("\n📋 Classification Report:")
print(classification_report(y_test, y_pred_best_model_2))


📋 Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.96      0.77      4656
           1       0.35      0.09      0.14      1422
           2       0.36      0.03      0.06      1302

    accuracy                           0.63      7380
   macro avg       0.45      0.36      0.32      7380
weighted avg       0.54      0.63      0.53      7380



In [ ]:
joblib.dump(best_model_v2, '/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/model/best_xgb_model.pkl')

['/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/model/best_xgb_model.pkl']

In [ ]:
# Load mô hình
loaded_model = joblib.load('/content/drive/MyDrive/thacsi/LuanVan/PredictStock/TA_FA/v2/model/best_xgb_model.pkl')

In [ ]:
y_pred_loaded_model = loaded_model.predict(X_test)

In [ ]:
print("🎯 Accuracy:", accuracy_score(y_test, y_pred_loaded_model))

🎯 Accuracy: 0.6285907859078591
